In [1]:
# importe os pacotes aqui
import requests
from pyspark.sql import SparkSession
from pyspark.sql.functions import col



In [ ]:
# url para o cojunto de dados 
url = "https://servicodados.ibge.gov.br/api/v3/agregados/1839/periodos/2022/variaveis/630?localidades=N1[all]"
response = requests.get(url)
data = response.json()

def extract_values(data):
    results = []
    for record in data[0]['resultados'][0]['series']:
        for year, value in record['serie'].items():
            results.append({
                'localidade': record['localidade']['nome'],
                'valor': int(value),
                'ano': int(year)
            })
    return results

spark = SparkSession.builder \
    .appName("Data Extraction and Transformation") \
    .getOrCreate()

df = spark.createDataFrame(extract_values(data))
df.show()



+----+----------+------+
| ano|localidade| valor|
+----+----------+------+
|2022|    Brasil|346105|
+----+----------+------+



In [9]:
# escreva o código de extração e transformação a partir daqui
df = df.withColumn('valor_dobrado', col('valor') * 2)
df.show()

+----+----------+------+-------------+
| ano|localidade| valor|valor_dobrado|
+----+----------+------+-------------+
|2022|    Brasil|346105|       692210|
+----+----------+------+-------------+



In [4]:
# teste do código
def test_data_extraction(df):
    assert df.count() > 0, "DataFrame está vazio"
    for column in ['localidade', 'valor', 'ano', 'valor_dobrado']:
        assert column in df.columns, f"Coluna '{column}' não encontrada"
    print("Todos os testes de extração e transformação passaram.")

test_data_extraction(df)

Todos os testes de extração e transformação passaram.
